In [ ]:
"""
LenderChange is a class used from interbank.py to control the change of lender in a bank.
   It contains different
    - Boltzman           using Boltzman probability to change
    - InitialStability   using a Barabási–Albert graph to initially assign relationships between banks
    - Preferential       using a Barabási-Albert with degree m to set up only a set o possible links between
                            banks. In each step, new links between are set up based on the initial graph
    - RestrictedMarket   using an Erdos Renyi graph with p=parameter['p']. This method does not allow the evolution in
                            lender for each bank, it replicates the situation after a crisis when banks do not credit
    - ShockedMarket      using an Erdos Renyi graph with p=parameter['p']. In each step a new random Erdos Renyi is
                            used
    - Small World        using a Watts and Strogatz algorithm with parameter['p'] and k=5
                            (Each node is joined with its k nearest neighbors in a ring topology)
@author: hector@bith.net
@date:   05/2023
"""
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import networkx as nx
import sys
import inspect
import json
import warnings

In [ ]:
def determine_algorithm(given_name: str = "default"):
    DEFAULT_METHOD = "Boltzman"

    if given_name == "default":
        warnings.warn(f"selected default method {DEFAULT_METHOD}")
        given_name = DEFAULT_METHOD
    if given_name == '?':
        for name, obj in inspect.getmembers(sys.modules[__name__]):
            if inspect.isclass(obj) and obj.__doc__:
                print("\t" + obj.__name__ + (" (default)" if name == DEFAULT_METHOD else '') + ':\n\t', obj.__doc__)
        sys.exit(0)
    else:
        for name, obj in inspect.getmembers(sys.modules[__name__]):
            if name.lower() == given_name.lower():
                if inspect.isclass(obj) and obj.__doc__:
                    return obj()
        print(f"not found lenderchange algorithm with name '{given_name}'")
        sys.exit(-1)

In [ ]:
node_positions = None
node_colors = None

In [ ]:
def draw(original_graph, new_guru_look_for=False, title=None, show=False):
    """ Draws the graph using a spring layout that reuses the previous one layout, to show similar position for
        the same ids of nodes along time. If the graph is undirected (Barabasi) then no """
    graph_to_draw = original_graph.copy()
    plt.clf()
    if title:
        plt.title(title)
    global node_positions, node_colors
    # if not node_positions:
    guru = None
    if node_positions is None:
        node_positions = nx.spring_layout(graph_to_draw, pos=node_positions)
    if not hasattr(original_graph, "type") and original_graph.is_directed():
        # guru should not have out edges, and surely by random graphs it has:
        guru, _ = find_guru(graph_to_draw)
        for (i, j) in list(graph_to_draw.out_edges(guru)):
            graph_to_draw.remove_edge(i, j)
    if hasattr(original_graph, "type") and original_graph.type == "barabasi_albert":
        graph_to_draw, guru = get_graph_from_guru(graph_to_draw.to_undirected())
    if hasattr(original_graph, "type") and original_graph.type == "erdos_renyi":
        for node in list(graph_to_draw.nodes()):
            if not graph_to_draw.edges(node) and not graph_to_draw.in_edges(node):
                graph_to_draw.remove_node(node)
        new_guru_look_for = True
    if not node_colors or new_guru_look_for:
        node_colors = []
        guru, guru_node_edges = find_guru(graph_to_draw)
        for node in graph_to_draw.nodes():
            if node == guru:
                node_colors.append('darkorange')
            elif __len_edges(graph_to_draw, node) == 0:
                node_colors.append('lightblue')
            elif __len_edges(graph_to_draw, node) == 1:
                node_colors.append('steelblue')
            else:
                node_colors.append('royalblue')
    if hasattr(original_graph, "type") and original_graph.type == "barabasi_pref":
        nx.draw(graph_to_draw, pos=node_positions, node_color=node_colors, with_labels=True)
    else:
        nx.draw(graph_to_draw, pos=node_positions, node_color=node_colors, arrowstyle='->',
                arrows=True, with_labels=True)

    if show:
        plt.show()
    return guru

def plot_degree_histogram(g, normalized=True):
    aux_y = nx.degree_histogram(g)
    aux_x = np.arange(0, len(aux_y)).tolist()
    n_nodes = g.number_of_nodes()
    if normalized:
        for i in range(len(aux_y)):
            aux_y[i] = aux_y[i] / n_nodes
    return aux_x, aux_y

In [ ]:
def save_graph_png(graph, description, filename, add_info=False):
    if add_info:
        if not description:
            description = ""
        description += " "+GraphStatistics.describe(graph)

    fig = plt.figure(layout="constrained")
    gs = gridspec.GridSpec(4, 4, figure=fig)
    fig.add_subplot(gs[:, :])
    guru = draw(graph, new_guru_look_for=True, title=description)
    plt.rcParams.update({'font.size': 6})

    # ax4 = fig.add_subplot(gs[-1, 0])
    # aux_x, aux_y = plot_degree_histogram(graph, False)
    # ax4.plot(aux_x, aux_y, 'o')
    # warnings.simplefilter("ignore")
    # ax4.set_xscale("log")
    # ax4.set_yscale("log")
    # warnings.resetwarnings()
    # ax4.set_xlabel('')
    # ax4.set_ylabel('')

    ax5 = fig.add_subplot(gs[-1, 0])
    aux_y = nx.degree_histogram(graph)
    aux_y.sort(reverse=True)
    aux_x = np.arange(0, len(aux_y)).tolist()
    ax5.loglog(aux_x, aux_y, 'o')
    ax5.set_xlabel('')
    ax5.set_ylabel('')

    plt.rcParams.update(plt.rcParamsDefault)
    plt.savefig(filename)
    plt.close('all')
    return guru

In [ ]:
def save_graph_json(graph, filename):
    if graph:
        graph_json = nx.node_link_data(graph)
        with open(filename, 'w') as f:
            json.dump(graph_json, f)

In [ ]:
def load_graph_json(filename):
    with open(filename, 'r') as f:
        graph_json = json.load(f)
        return nx.node_link_graph(graph_json)

In [ ]:
def __len_edges(graph, node):
    if hasattr(graph, "in_edges"):
        return len(graph.in_edges(node))
    else:
        return len(graph.edges(node))

In [ ]:
def find_guru(graph):
    """It returns the guru ID and also a color_map with red for the guru, lightblue if weight<max/2 and blue others """
    guru_node = None
    guru_node_edges = 0
    for node in graph.nodes():
        edges_node = __len_edges(graph, node)
        if guru_node_edges < edges_node:
            guru_node_edges = edges_node
            guru_node = node
    return guru_node, guru_node_edges

In [ ]:
class GraphStatistics:
    @staticmethod
    def giant_component_size(graph):
        """weakly connected componentes of the directed graph using Tarjan's algorithm:
           https://en.wikipedia.org/wiki/Tarjan%27s_strongly_connected_components_algorithm"""
        if graph.is_directed():
            return len(max(nx.weakly_connected_components(graph), key=len))
        else:
            return len(max(nx.connected_components(graph), key=len))

    @staticmethod
    def clustering_coeff(graph):
        """clustering coefficient 0..1, 1 for totally connected graphs, and 0 for totally isolated
           if ~0 then a small world"""
        try:
            return nx.average_clustering(graph, count_zeros=True)
        except ZeroDivisionError:
            return 0

    @staticmethod
    def communities(graph):
        """number of communities using greedy modularity maximization"""
        return nx.community.greedy_modularity_communities(graph)

    @staticmethod
    def describe(graph):
        clustering = GraphStatistics.clustering_coeff(graph)
        if clustering > 0 and clustering < 1:
            clustering = f"clus_coef={clustering:5.3f}"
        else:
            clustering = f"clus_coef={clustering}"
        return (f"giant={GraphStatistics.giant_component_size(graph)} " +
                clustering +
                f" comm={len(GraphStatistics.communities(graph))}")

In [ ]:
def __get_graph_from_guru(input_graph, output_graph, current_node, previous_node):
    """ It generates a new graph starting from the guru"""
    if __len_edges(input_graph, current_node) > 1:
        for (_, destination) in input_graph.edges(current_node):
            if destination != previous_node:
                __get_graph_from_guru(input_graph, output_graph, destination, current_node)
    if previous_node is not None:
        output_graph.add_edge(current_node, previous_node)

In [ ]:
def get_graph_from_guru(input_graph):
    guru, _ = find_guru(input_graph)
    output_graph = nx.DiGraph()
    __get_graph_from_guru(input_graph, output_graph, guru, None)
    return output_graph, guru

In [ ]:
def from_graph_to_array_banks(banks_graph, this_model):
    """ From the graph to a lender for each possible bank (or None if no links in the graph)"""
    for node in banks_graph:
        if banks_graph.out_edges(node):
            edges = list(banks_graph.out_edges(node))
            edge_selected = random.randrange(len(edges))
            this_model.banks[node].lender = edges[edge_selected][1]
        else:
            this_model.banks[node].lender = None

In [ ]:
# ---------------------------------------------------------
# prototype
class LenderChange:
    GRAPH_NAME = ""

    def __init__(self):
        self.parameter = {}
        self.initial_graph_file = None

    def initialize_bank_relationships(self, this_model):
        """ Call once at initilize() model """
        pass

    def step_setup_links(self, this_model):
        """ Call at the end of each step """
        pass

    def change_lender(self, this_model, bank, t):
        """ Call at the end of each step before going to the next"""
        pass

    def new_lender(self, this_model, bank):
        """ Describes the mechanism of change"""
        pass

    def set_initial_graph_file(self, lc_ini_graph_file):
        if lc_ini_graph_file:
            self.initial_graph_file = lc_ini_graph_file

    def describe(self):
        return ""

    def check_parameter(self, name, value):
        """ Called after set_parameter() to verify that the necessary parameters are set """
        return False

    def set_parameter(self, name, value):
        if not value is None:
            if self.check_parameter(name, value):
                self.parameter[name] = value
            else:
                print(f"error with parameter '{name}' for {self.__class__.__name__}")
                sys.exit(-1)

---------------------------------------------------------

In [ ]:
class Boltzman(LenderChange):
    """It chooses randomly a lender for each bank and in each step changes using Boltzman's probability
         Also it has parameter γ to control the change [0..1], 0=Boltzmann only, 1 everyone will move randomly
    """
    # parameter to control the change of guru: 0 then Boltzmann only, 1 everyone will move randomly
    gamma: float = 0.5  # [0..1] gamma
    CHANGE_LENDER_IF_HIGHER = 0.5

    def initialize_bank_relationships(self, this_model):
        if self.initial_graph_file:
            graph = load_graph_json(self.initial_graph_file)
            from_graph_to_array_banks(graph, this_model)
        if this_model.export_datafile:
            this_model.statistics.get_graph(0)

    def change_lender(self, this_model, bank, t):
        """ It uses γ but only after t=20, at the beginning only Boltzmann"""
        possible_lender = self.new_lender(this_model, bank)
        if possible_lender is None:
            possible_lender_mi = 0
        else:
            possible_lender_mi = this_model.banks[possible_lender].mu
        if bank.getLender() is None:
            current_lender_mi = 0
        else:
            current_lender_mi = bank.getLender().mu

        # we can now break old links and set up new lenders, using probability P
        # (equation 8)
        boltzmann = 1 / (1 + math.exp(-this_model.config.beta * (possible_lender_mi - current_lender_mi)))

        if t < 20:
            # bank.P = 0.35
            # bank.P = random.random()
            bank.P = boltzmann
            # option a) bank.P initially 0.35
            # option b) bank.P randomly
            # option c) with t<=20 boltzmann, and later, stabilize it
        else:
            bank.P_yesterday = bank.P
            # gamma is not sticky/loyalty, persistence of the previous attitude
            bank.P = self.gamma * bank.P_yesterday + (1 - self.gamma) * boltzmann

        if bank.P >= self.CHANGE_LENDER_IF_HIGHER:
            text_to_return = f"{bank.getId()} new lender is #{possible_lender} from #{bank.lender} with %{bank.P:.3f}"
            bank.lender = possible_lender
        else:
            text_to_return = f"{bank.getId()} maintains lender #{bank.lender} with %{1 - bank.P:.3f}"
        return text_to_return

    def new_lender(self, this_model, bank):
        """ It gives to the bank a random new lender. It's used initially and from change_lender() """
        # r_i0 is used the first time the bank is created:
        if bank.lender is None:
            bank.rij = np.full(this_model.config.N, this_model.config.r_i0, dtype=float)
            bank.rij[bank.id] = 0
            bank.r = this_model.config.r_i0
            bank.mu = 0
            bank.asset_i = 0
            bank.asset_j = 0
            bank.asset_equity = 0
            # if it's just created, only not to be ourselves is enough
            new_value = random.randrange(this_model.config.N - 1)
        else:
            # if we have a previous lender, new should not be the same
            new_value = random.randrange(this_model.config.N - 2 if this_model.config.N > 2 else 1)

        if this_model.config.N == 2:
            new_value = 1 if bank.id == 0 else 0
        else:
            if new_value >= bank.id:
                new_value += 1
                if bank.lender is not None and new_value >= bank.lender:
                    new_value += 1
            else:
                if bank.lender is not None and new_value >= bank.lender:
                    new_value += 1
                    if new_value >= bank.id:
                        new_value += 1
        return new_value

    def describe(self):
        return f"($\\gamma={self.gamma} and change if >{self.CHANGE_LENDER_IF_HIGHER})$"

In [ ]:
class InitialStability(Boltzman):
    """ We define a Barabási–Albert graph with 1 edges for each node, and we convert it to a directed graph.
          It is used for initially have more stable links between banks
    """

    CHANGE_LENDER_IF_HIGHER = 0.8
    GRAPH_NAME = 'barabasi_albert'

    def __create_directed_graph_from_barabasi_albert(self, barabasi_albert, result, current_node, previous_node):
        if len(barabasi_albert.edges(current_node)) > 1:
            for (_, destination) in barabasi_albert.edges(current_node):
                if destination != previous_node:
                    self.__create_directed_graph_from_barabasi_albert(barabasi_albert, result,
                                                                      destination, current_node)
                # if not previous_node:
                #    result.add_edge(destination, current_node)
        if previous_node is not None:
            result.add_edge(current_node, previous_node)

    def initialize_bank_relationships(self, this_model):
        if self.initial_graph_file:
            self.banks_graph = load_graph_json(self.initial_graph_file)
            description = f"{self.GRAPH_NAME} from file {self.initial_graph_file}"
        else:
            self.banks_graph, _ = get_graph_from_guru(nx.barabasi_albert_graph(this_model.config.N, 1))
            description = f"{self.GRAPH_NAME} m=1 {GraphStatistics.describe(self.banks_graph)}"
        self.banks_graph.type = self.GRAPH_NAME
        if this_model.export_datafile:
            save_graph_png(self.banks_graph, description,
                           this_model.statistics.get_export_path(this_model.export_datafile, f"_{self.GRAPH_NAME}.png"))
            save_graph_json(self.banks_graph,
                            this_model.statistics.get_export_path(this_model.export_datafile,
                                                                  f"_{self.GRAPH_NAME}.json"))
        from_graph_to_array_banks(self.banks_graph, this_model)
        return self.banks_graph

In [ ]:
class RestrictedMarket(LenderChange):
    """ Using an Erdos Renyi graph with p=parameter['p']. This method does not allow the evolution in
          lender for each bank, it replicates the situation after a crisis when banks do not credit

    """

    GRAPH_NAME = "erdos_renyi"

    def check_parameter(self, name, value):
        if name == 'p':
            if isinstance(value, float) and 0 <= value <= 1:
                return True
            else:
                print("value for 'p' should be a float number >= 0 and <= 1")
                return False
        else:
            return False

    def initialize_bank_relationships(self, this_model):
        """ It creates a Erdos Renyi graph with p defined in parameter['p']. No changes in relationships before end"""
        if self.initial_graph_file:
            self.banks_graph = load_graph_json(self.initial_graph_file)
            description = f"erdos_renyi from file {self.initial_graph_file}"
        else:
            self.banks_graph = nx.erdos_renyi_graph(this_model.config.N, self.parameter['p'], directed=True)
            description = f"erdos_renyi p={self.parameter['p']:5.3} {GraphStatistics.describe(self.banks_graph)}"
        self.banks_graph.type = self.GRAPH_NAME
        if this_model.export_datafile:
            save_graph_png(self.banks_graph, description,
                           this_model.statistics.get_export_path(this_model.export_datafile, f"_{self.GRAPH_NAME}.png"))
            save_graph_json(self.banks_graph,
                            this_model.statistics.get_export_path(this_model.export_datafile,
                                                                  f"_{self.GRAPH_NAME}.json"))

        from_graph_to_array_banks(self.banks_graph, this_model)
        if this_model.export_datafile:
            this_model.statistics.get_graph(0)
        return self.banks_graph

    def new_lender(self, this_model, bank):
        """ In this LenderChange we never change of lender """
        bank.rij = np.full(this_model.config.N, this_model.config.r_i0, dtype=float)
        bank.rij[bank.id] = 0
        bank.r = this_model.config.r_i0
        bank.mu = 0
        return bank.lender

    def change_lender(self, this_model, bank, t):
        """ In this LenderChange we never change of lender """
        bank.P = 0
        return f"{bank.getId()} maintains lender #{bank.lender} with %1 (ShockedMarket)"

In [ ]:
class Preferential(Boltzman):
    """ Using a Barabasi with grade m we restrict to those relations the possibilities to obtain an outgoing link
          (a lender). To improve the specialization of banks, granting 3*C0 to the guru, 2*C0 to its neighbours
          and C to the others. To balance those with 2C0 and 3C0, we will reduce D
        In each step, we change the lender using the base self.banks_graph_full
    """
    banks_graph = None
    guru = None
    GRAPH_NAME = "barabasi_pref"

    def check_parameter(self, name, value):
        if name == 'm':
            if isinstance(value, int) and 1 <= value:
                return True
            else:
                print("value for 'm' should be an integer >= 1")
                return False
        else:
            return False

    def initialize_bank_relationships(self, this_model):

        if self.initial_graph_file:
            self.banks_graph_full = load_graph_json(self.initial_graph_file)
            description = f"{self.GRAPH_NAME} from file {self.initial_graph_file}"
        else:
            self.banks_graph_full = nx.barabasi_albert_graph(this_model.config.N, self.parameter['m'])
            description = (f"{self.GRAPH_NAME} m={self.parameter['m']:5.3f} "
                           f"{GraphStatistics.describe(self.banks_graph_full)}")
        self.banks_graph_full.type = self.GRAPH_NAME
        if this_model.export_datafile:
            self.guru = save_graph_png(self.banks_graph_full, description,
                                       this_model.statistics.get_export_path(this_model.export_datafile,
                                                                             f"_{self.GRAPH_NAME}.png"))
        else:
            self.guru, _ = find_guru(self.banks_graph_full)
        self.full_barabasi_extract_random_directed(this_model)
        self.prize_for_good_banks(this_model)
        if this_model.export_datafile:
            save_graph_json(self.banks_graph,
                            this_model.statistics.get_export_path(this_model.export_datafile,
                                                                  f"_{self.GRAPH_NAME}.json"))
        return self.banks_graph

    def prize_for_good_banks(self, this_model):
        this_model.banks[self.guru].D -= this_model.banks[self.guru].C * 2
        this_model.banks[self.guru].C *= 3
        for (_, node) in self.banks_graph_full.edges(self.guru):
            this_model.banks[node].D -= this_model.banks[node].C
            this_model.banks[node].C *= 2

    def step_setup_links(self, this_model):
        self.full_barabasi_extract_random_directed(this_model)


    def full_barabasi_extract_random_directed(self, this_model, current_node=None):
        if current_node is None:
            self.banks_graph = nx.DiGraph()
            self.banks_graph.type = self.GRAPH_NAME
            current_node = self.guru
        edges = list(self.banks_graph_full.edges(current_node))
        self.banks_graph.add_node(current_node)
        for (_, destination) in edges[:]:
            if destination not in self.banks_graph.nodes():
                self.full_barabasi_extract_random_directed(this_model, destination)
        try:
            edges.remove((current_node, this_model.banks[current_node].lender))
        except ValueError:
            pass
        candidate = None
        while candidate is None and edges:
            candidate = random.choice(edges)[1]
            if (candidate, current_node) in self.banks_graph.edges():
                edges.remove((current_node, candidate))
                candidate = None
        if candidate is None:
            if this_model.banks[current_node].lender is not None:
                self.banks_graph.add_edge(current_node, this_model.banks[current_node].lender)
        else:
            self.banks_graph.add_edge(current_node, candidate)
        return current_node

    def new_lender(self, this_model, bank):
        if bank.lender is None:
            bank.rij = np.full(this_model.config.N, this_model.config.r_i0, dtype=float)
            bank.rij[bank.id] = 0
            bank.r = this_model.config.r_i0
            bank.mu = 0
        if self.banks_graph and self.banks_graph.out_edges() and self.banks_graph.out_edges(bank.id):
            return list(self.banks_graph.out_edges(bank.id))[0][1]
        else:
            return None

    def describe(self):
        return f"($\\gamma={self.gamma} and change if >{self.CHANGE_LENDER_IF_HIGHER})$"

In [ ]:
class ShockedMarket(LenderChange):
    """ Using an Erdos Renyi graph with p=parameter['p']. This method replicate RestrictedMarket
        but using a new network relationship between banks, but always with same p. So the links
        in t=i are destroyed and new aleatory links in t=i+1 are created using a new Erdos Renyi
        graph.
    """

    GRAPH_NAME = "erdos_renyi"

    def check_parameter(self, name, value):
        if name == 'p':
            if isinstance(value, float) and 0 <= value <= 1:
                return True
            else:
                print(f"value for 'p' should be a float number >= 0 and <= 1: {value}")
                return False
        else:
            return False

    def step_setup_links(self, this_model):
        self.banks_graph = nx.erdos_renyi_graph(this_model.config.N, self.parameter['p'], directed=True)
        self.banks_graph.type = self.GRAPH_NAME
        from_graph_to_array_banks(self.banks_graph, this_model)

    def initialize_bank_relationships(self, this_model):
        """ It creates a Erdos Renyi graph with p defined in parameter['p']. No changes in relationships till the end"""
        if self.initial_graph_file:
            self.banks_graph = load_graph_json(self.initial_graph_file)
            description = f"{self.GRAPH_NAME} from file {self.initial_graph_file}"
        else:
            self.banks_graph = nx.erdos_renyi_graph(this_model.config.N, self.parameter['p'], directed=True)
            description = f"{self.GRAPH_NAME} p={self.parameter['p']:5.3} {GraphStatistics.describe(self.banks_graph)}"
        self.banks_graph.type = self.GRAPH_NAME
        if this_model.export_datafile:
            save_graph_png(self.banks_graph, description,
                           this_model.statistics.get_export_path(this_model.export_datafile, f"_{self.GRAPH_NAME}.png"))
            save_graph_json(self.banks_graph,
                            this_model.statistics.get_export_path(this_model.export_datafile,
                                                                  f"_{self.GRAPH_NAME}.json"))

        from_graph_to_array_banks(self.banks_graph, this_model)
        if this_model.export_datafile:
            this_model.statistics.get_graph(0)
        return self.banks_graph

    def new_lender(self, this_model, bank):
        """ In this LenderChange we never change of lender """
        bank.rij = np.full(this_model.config.N, this_model.config.r_i0, dtype=float)
        bank.rij[bank.id] = 0
        bank.r = this_model.config.r_i0
        bank.asset_i = 0
        bank.asset_j = 0
        bank.asset_equity = 0
        bank.mu = 0
        return bank.lender

    def change_lender(self, this_model, bank, t):
        """ In this LenderChange we never change of lender """
        bank.P = 0
        return f"{bank.getId()} maintains lender #{bank.lender} with %1 (ShockedMarket)"

In [ ]:
class SmallWorld(ShockedMarket):
    """ SmallWorld implementation using Watts Strogatz
    """

    GRAPH_NAME = 'small_world'
    # Each node is joined with its k nearest neighbors in a ring topology (Watts Strogatz parameter):
    K = 5

    @staticmethod
    def create_directed_graph_from_watts_strogatz(graph, result=None, pending=None, current_node=None):
        # a) first time: look for extremes of the graph and use current_node to create links from them:
        if result is None:
            result = nx.DiGraph()
            pending = list(graph.nodes())
            # in each node with only one link, we start from it:
            for node in graph.nodes():
                edges = list(graph.edges(node))
                if len(edges) == 1:
                    SmallWorld.create_directed_graph_from_watts_strogatz(graph, result, pending, node)
            # no nodes with only one option: we choose an arbitrary node to start:
            if result.edges() == 0:
                random_node = random.choice(graph.nodes())
                SmallWorld.create_directed_graph_from_watts_strogatz(graph, result, pending, random_node)
            SmallWorld.create_directed_graph_from_watts_strogatz(graph, result, pending)
        elif current_node:
            # b) nodes with an incoming link that we should follow (there's only that option)
            source = current_node
            destination = list(graph.edges(source))[0][1]
            while not destination is None:
                if destination not in result or len(result.in_edges(destination)) == 0:
                    result.add_edge(source, destination)
                    pending.remove(source)
                    edges = list(graph.edges(destination))
                    edges.remove((destination, source))
                    source = destination
                    if len(edges) == 1:
                        destination = edges[0][1]
                    else:
                        destination = None
                else:
                    destination = None
        elif pending:
            # c) items with >1 node
            for source in pending:
                edges = list(graph.edges(source))
                if result.has_node(source):
                    for in_edges in result.in_edges(source):
                        if (in_edges[1], in_edges[0]) in edges:
                            edges.remove((in_edges[1], in_edges[0]))
                if edges:
                    new_link = random.choice(edges)
                    result.add_edge(new_link[0], new_link[1])
                else:
                    result.add_node(source)
        return result

    def check_parameter(self, name, value):
        if name == 'p':
            if isinstance(value, float) and 0 <= value <= 1:
                return True
            else:
                print("value for 'p' should be a float number >= 0 and <= 1")
                return False
        else:
            return False

    def step_setup_links(self, this_model):
        self.banks_graph = SmallWorld.create_directed_graph_from_watts_strogatz(self.banks_smallworld)
        self.banks_graph.type = self.GRAPH_NAME
        from_graph_to_array_banks(self.banks_graph, this_model)

    def initialize_bank_relationships(self, this_model):
        """ It creates a small world graph using Watts Strogatz. It's indirected and we directed it using an own
                algorithm which guarantees 1 output link only for each node (only a lender for each bank)
            """
        if self.initial_graph_file:
            self.banks_smallworld = load_graph_json(self.initial_graph_file)
            self.description = f"{self.GRAPH_NAME} from file {self.initial_graph_file}"
        else:
            self.banks_smallworld = nx.watts_strogatz_graph(this_model.config.N, self.K, self.parameter['p'])
            self.description = f"{self.GRAPH_NAME} p={self.parameter['p']:5.3}"
        self.banks_graph = SmallWorld.create_directed_graph_from_watts_strogatz(self.banks_smallworld)
        self.description += f"{GraphStatistics.describe(self.banks_graph)}"
        self.banks_graph.type = self.GRAPH_NAME
        if this_model.export_datafile:
            save_graph_png(self.banks_graph, self.description,
                           this_model.statistics.get_export_path(this_model.export_datafile, f"_{self.GRAPH_NAME}.png"))
            save_graph_json(self.banks_graph,
                            this_model.statistics.get_export_path(this_model.export_datafile,
                                                                  f"_{self.GRAPH_NAME}.json"))
        from_graph_to_array_banks(self.banks_graph, this_model)
        return self.banks_graph

In [ ]:
"""
Generates a simulation of an interbank network following the rules described in paper
  Reinforcement Learning Policy Recommendation for Interbank Network Stability
  from Gabrielle and Alessio

  You can use it interactively, but if you import it, the process will be:
    # model = Model()
    #   # step by step:
    #   model.enable_backward()
    #   model.forward() # t=0 -> t=1
    #   model.backward() : reverts the last step (when executed
    #   # all in a loop:
    #   model.simulate_full()

@author: hector@bith.net
@date:   04/2023
"""
import copy
import random
import logging
import argparse
import numpy as np
import networkx as nx
import sys
import os
from PIL import Image
import matplotlib.pyplot as plt

import pandas as pd
import lxml.etree
import lxml.builder
import gzip


class Config:
    """
    Configuration parameters for the interbank network
    """
    T: int = 1000  # time (1000)
    N: int = 50  # number of banks (50)

    # not used in this implementation:
    # ȓ: float  = 0.02     # percentage reserves (at the moment, no R is used)
    # đ: int    = 1        # number of outgoing links allowed

    # shocks parameters:
    mi: float = 0.7  # mi µ
    omega: float = 0.55  # omega ω

    # Lender's change mechanism
    lender_change: LenderChange = None

    # screening costs
    phi: float = 0.025  # phi Φ
    ji: float = 0.015  # ji Χ

    # liquidation cost of collateral
    xi: float = 0.3  # xi ξ
    ro: float = 0.3  # ro ρ fire sale cost

    beta: float = 5  # β beta intensity of breaking the connection (5)
    alfa: float = 0.1  # α alfa below this level of E or D, we will bankrupt the bank

    # banks initial parameters
    # L + C + (R) = D + E
    L_i0: float = 120  # long term assets
    C_i0: float = 30  # capital
    D_i0: float = 135  # deposits
    E_i0: float = 15  # equity
    r_i0: float = 0.02  # initial rate

    # if enabled and != [] the values of t in the array (for instance [150,350]) will generate
    # a graph with the relations of the firms. If * all the instants will generate a graph, and also an animated gif
    # with the results
    GRAPHS_MOMENTS = []

    # what elements are in the results.csv file, and also which are plot.
    # 1 if also plot, 0 not to plot:
    ELEMENTS_STATISTICS = {'B': True, 'liquidity': True, 'interest_rate': True, 'asset_i': True, 'asset_j': True,
                           'equity': True, 'bankruptcy': True, 'credit_channels': True, 'P': True, 'best_lender': True,
                           'policy': False, 'fitness': False, 'best_lender_clients': False,
                           'rationing': False, 'leverage': False, 'loans': False, 'num_lenders': False,
                           'num_borrowers': False, 'prob_bankruptcy': False}

    def __str__(self):
        description = sys.argv[0] if __name__ == '__main__' else ''
        for attr in dir(self):
            value = getattr(self, attr)
            if isinstance(value, int) or isinstance(value, float):
                description += f" {attr}={value}"
        return description

In [ ]:

class Statistics:
    bankruptcy = []
    best_lender = []
    best_lender_clients = []
    credit_channels = []
    liquidity = []
    policy = []
    interest_rate = []
    incrementD = []
    fitness = []
    rationing = []
    leverage = []
    loans = []
    asset_i = []
    asset_j = []
    equity = []
    P = []
    B = []
    num_borrowers = []
    num_lenders = []
    prob_bankruptcy = []
    model = None
    graphs = {}
    graphs_pos = None

    plot_format = None
    graph_format = ".svg"
    output_format = ".gdt"
    create_gif = False

    OUTPUT_DIRECTORY = "output"
    NUMBER_OF_ITEMS_IN_ANIMATED_GRAPH = 40

    def __init__(self, in_model):
        self.model = in_model

    def set_gif_graph(self, gif_graph):
        if gif_graph:
            self.create_gif = True

    def reset(self, output_directory=None):
        if output_directory:
            self.OUTPUT_DIRECTORY = output_directory
        if not os.path.isdir(self.OUTPUT_DIRECTORY):
            os.mkdir(self.OUTPUT_DIRECTORY)
        self.bankruptcy = np.zeros(self.model.config.T, dtype=int)
        self.best_lender = np.full(self.model.config.T, -1, dtype=int)
        self.best_lender_clients = np.zeros(self.model.config.T, dtype=int)
        self.credit_channels = np.zeros(self.model.config.T, dtype=int)
        self.num_borrowers = np.zeros(self.model.config.T, dtype=int)
        self.prob_bankruptcy = np.zeros(self.model.config.T, dtype=float)
        self.num_lenders = np.zeros(self.model.config.T, dtype=int)
        self.fitness = np.zeros(self.model.config.T, dtype=float)
        self.interest_rate = np.zeros(self.model.config.T, dtype=float)
        self.asset_i = np.zeros(self.model.config.T, dtype=float)
        self.asset_j = np.zeros(self.model.config.T, dtype=float)
        self.equity = np.zeros(self.model.config.T, dtype=float)
        self.incrementD = np.zeros(self.model.config.T, dtype=float)
        self.liquidity = np.zeros(self.model.config.T, dtype=float)
        self.rationing = np.zeros(self.model.config.T, dtype=float)
        self.leverage = np.zeros(self.model.config.T, dtype=float)
        self.policy = np.zeros(self.model.config.T, dtype=float)
        self.P = np.zeros(self.model.config.T, dtype=float)
        self.P_max = np.zeros(self.model.config.T, dtype=float)
        self.P_min = np.zeros(self.model.config.T, dtype=float)
        self.P_std = np.zeros(self.model.config.T, dtype=float)
        self.B = np.zeros(self.model.config.T, dtype=float)
        self.loans = np.zeros(self.model.config.T, dtype=float)

    def compute_credit_channels_and_best_lender(self):
        lenders = {}
        for bank in self.model.banks:
            if bank.lender:
                if bank.lender in lenders:
                    lenders[bank.lender] += 1
                else:
                    lenders[bank.lender] = 1
        best = -1
        best_value = -1
        for lender in lenders.keys():
            if lenders[lender] > best_value:
                best = lender
                best_value = lenders[lender]

        self.best_lender[self.model.t] = best
        self.best_lender_clients[self.model.t] = best_value
        # self.credit_channels is updated directly at the moment the credit channel is set up
        # during Model.do_loans()

    def compute_interest_loans_leverage(self):
        num_of_banks_with_lenders = 0
        num_of_banks_with_borrowers = 0
        sum_of_interests_rates = 0

        sum_of_asset_i = 0
        sum_of_asset_j = 0
        sum_of_equity = 0
        sum_of_loans = 0
        sum_of_leverage = 0
        maxE = 0

        for bank in self.model.banks:
            if bank.getLoanInterest() is not None and bank.l > 0:
                sum_of_interests_rates += bank.getLoanInterest()
                sum_of_asset_i += bank.asset_i
                sum_of_asset_j += bank.asset_j
                sum_of_equity += bank.E
                num_of_banks_with_lenders += 1
                sum_of_loans += bank.l
                sum_of_leverage += (bank.l / bank.E)
                if bank.E > maxE:
                    maxE = bank.E
            if bank.activeBorrowers:
                num_of_banks_with_borrowers += 1

        if num_of_banks_with_lenders:
            avg_prob_bankruptcy = 0
            num_elements = 0
            if maxE > 0:  #TODO
                for bank in self.model.banks:
                    if bank.getLoanInterest() is not None and bank.l > 0:
                        avg_prob_bankruptcy += (1 - bank.E / maxE)
                        num_elements += 1
                avg_prob_bankruptcy = avg_prob_bankruptcy / num_elements

            self.interest_rate[self.model.t] = sum_of_interests_rates / num_of_banks_with_lenders
            self.asset_i[self.model.t] = sum_of_asset_i / num_of_banks_with_lenders
            self.asset_j[self.model.t] = sum_of_asset_j / num_of_banks_with_lenders
            self.equity[self.model.t] = sum_of_equity / num_of_banks_with_lenders
            self.loans[self.model.t] = sum_of_loans / num_of_banks_with_lenders
            self.leverage[self.model.t] = sum_of_leverage / num_of_banks_with_lenders
            self.num_lenders[self.model.t] = num_of_banks_with_lenders
            self.prob_bankruptcy[self.model.t] = avg_prob_bankruptcy
        else:
            self.interest_rate[self.model.t] = np.nan
            self.asset_j[self.model.t] = np.nan
            self.asset_j[self.model.t] = np.nan
            self.equity[self.model.t] = np.nan
            self.loans[self.model.t] = np.nan
            self.leverage[self.model.t] = np.nan
            self.num_lenders[self.model.t] = 0
            self.prob_bankruptcy[self.model.t] = 0
        self.num_borrowers[self.model.t] = num_of_banks_with_borrowers

    def compute_liquidity(self):
        self.liquidity[self.model.t] = sum(map(lambda x: x.C, self.model.banks))

    def compute_fitness(self):
        self.fitness[self.model.t] = sum(map(lambda x: x.mu, self.model.banks)) / self.model.config.N

    def compute_policy(self):
        self.policy[self.model.t] = self.model.eta

    def compute_bad_debt(self):
        self.B[self.model.t] = sum(map(lambda x: x.B, self.model.banks))

    def compute_rationing(self):
        self.rationing[self.model.t] = sum(map(lambda x: x.rationing, self.model.banks))

    def compute_probability_of_lender_change(self):
        probabilities = [bank.P for bank in self.model.banks]
        self.P[self.model.t] = sum(probabilities) / self.model.config.N
        self.P_max[self.model.t] = max(probabilities)
        self.P_min[self.model.t] = min(probabilities)
        self.P_std[self.model.t] = np.std(probabilities)

    def export_data(self, export_datafile=None, export_description=None, generate_plots=True):
        if export_datafile:
            self.save_data(export_datafile, export_description)
            if generate_plots:
                self.get_plots(export_datafile)
        if Utils.is_notebook() or Utils.is_spyder():
            self.get_plots(None)

    def get_graph(self, t):
        """
        Extracts from the model the graph that corresponds to the network in this instant
        """
        if 'unittest' in sys.modules.keys():
            return None
        else:
            self.graphs[t] = nx.DiGraph(directed=True)
            for bank in self.model.banks:
                if bank.lender:
                    self.graphs[t].add_edge(bank.id, bank.lender)
            draw(self.graphs[t], new_guru_look_for=True, title=f"t={t}")
            if Utils.is_spyder():
                plt.show()
                filename = None
            else:
                filename = sys.argv[0] if self.model.export_datafile is None else self.model.export_datafile
                filename = self.get_export_path(filename, f"_{t}{self.graph_format}")
                plt.savefig(filename)
            plt.close()
            return filename

    def define_plot_format(self, plot_format):
        match plot_format.lower():
            case 'none':
                self.plot_format = None
            case 'svg':
                self.plot_format = '.svg'
            case 'png':
                self.plot_format = '.png'
            case 'gif':
                self.plot_format = '.gif'
            case 'pdf':
                self.plot_format = '.pdf'
            case 'agr':
                self.plot_format = '.agr'
            case _:
                print(f'Invalid plot file format: {plot_format}')
                sys.exit(-1)

    def define_output_format(self, output_format):
        match output_format.lower():
            case 'both':
                self.output_format = '.both'
            case "gdt":
                self.output_format = '.gdt'
            case 'csv':
                self.output_format = '.csv'
            case 'txt':
                self.output_format = '.txt'
            case _:
                print(f'Invalid output file format: {output_format}')
                sys.exit(-1)

    def create_gif_with_graphs(self, list_of_files):
        if len(list_of_files) == 0 or not self.create_gif:
            return
        else:
            if len(list_of_files) > self.NUMBER_OF_ITEMS_IN_ANIMATED_GRAPH:
                positions_of_images = len(list_of_files) / self.NUMBER_OF_ITEMS_IN_ANIMATED_GRAPH
            else:
                positions_of_images = 1
            filename_output = sys.argv[0] if self.model.export_datafile is None else self.model.export_datafile
            filename_output = self.get_export_path(filename_output, '.gif')
            images = []
            for idx, image_file in enumerate(list_of_files):
                # if more >40 images, only those that are divisible by 40 are incorporated:
                if not (idx % positions_of_images == 0):
                    continue
                images.append(Image.open(image_file))
            images[0].save(fp=filename_output, format='GIF', append_images=images[1:],
                           save_all=True, duration=100, loop=0)

    def get_export_path(self, filename, ending_name=''):
        # we ensure that the output goes to OUTPUT_DIRECTORY:
        if not os.path.dirname(filename):
            filename = f"{self.OUTPUT_DIRECTORY}/{filename}"
        path, extension = os.path.splitext(filename)
        # if there is an ending_name it means that we don't want the output.csv, we are using the
        # function to generate a plot_file, for instance:
        if ending_name:
            return path + ending_name
        else:
            # we ensure that the output goes with the correct extension:
            return path + self.output_format.lower()

    def __generate_csv_or_txt(self, export_datafile, header, delimiter):
        with open(export_datafile, 'w', encoding="utf-8") as savefile:
            for line_header in header:
                savefile.write(f"# {line_header}\n")
            savefile.write(f"# pd.read_csv('file{self.output_format}',header={len(header) + 1}',"
                           f" delimiter='{delimiter}')\nt")
            for element_name, _ in self.enumerate_results():
                savefile.write(f"{delimiter}{element_name}")
            savefile.write("\n")
            for i in range(self.model.config.T):
                savefile.write(f"{i}")
                for _, element in self.enumerate_results():
                    savefile.write(f"{delimiter}{element[i]}")
                savefile.write(f"\n")

    def __generate_gdt(self, export_datafile, header):
        E = lxml.builder.ElementMaker()
        GRETLDATA = E.gretldata
        DESCRIPTION = E.description
        VARIABLES = E.variables
        VARIABLE = E.variable
        OBSERVATIONS = E.observations
        OBS = E.obs
        variables = VARIABLES(count=f"{sum(1 for _ in self.enumerate_results())}")
        for variable_name, _ in self.enumerate_results():
            if variable_name == 'leverage':
                variable_name += "_"
            variables.append(VARIABLE(name=f"{variable_name}"))

        observations = OBSERVATIONS(count=f"{self.model.config.T}", labels="false")
        for i in range(self.model.config.T):
            string_obs = ''
            for _, variable in self.enumerate_results():
                string_obs += f"{variable[i]}  "
            observations.append(OBS(string_obs))
        header_text = ""
        for item in header:
            header_text += item + " "
        gdt_result = GRETLDATA(
            DESCRIPTION(header_text),
            variables,
            observations,
            version="1.4", name='prueba', frequency="special:1", startobs="1",
            endobs=f"{self.model.config.T}", type="time-series"
        )
        with gzip.open(self.get_export_path(export_datafile), 'w') as output_file:
            output_file.write(
                b'<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE gretldata SYSTEM "gretldata.dtd">\n')
            output_file.write(
                lxml.etree.tostring(gdt_result, pretty_print=True, encoding=str).encode('ascii'))

    @staticmethod
    def __transform_line_from_string(line_with_values):
        items = []
        for i in line_with_values.replace("  ", " ").strip().split(" "):
            try:
                items.append(int(i))
            except ValueError:
                items.append(float(i))
        return items

    @staticmethod
    def read_gdt(filename):
        tree = lxml.etree.parse(filename)
        root = tree.getroot()
        children = root.getchildren()
        values = []
        columns = []
        if len(children) == 3:
            # children[0] = description
            # children[1] = variables
            # children[2] = observations
            for variable in children[1].getchildren():
                column_name = variable.values()[0].strip()
                if column_name == 'leverage_':
                    column_name = 'leverage'
                columns.append(column_name)
            for value in children[2].getchildren():
                values.append(Statistics.__transform_line_from_string(value.text))
        if columns and values:
            return pd.DataFrame(columns=columns, data=values)
        else:
            return pd.Dataframe()

    def save_data(self, export_datafile=None, export_description=None):
        if export_datafile:
            if export_description:
                header = [f"{export_description}"]
            else:
                header = [f"{__name__} T={self.model.config.T} N={self.model.config.N}"]
            if self.output_format.lower() == '.both':
                self.output_format = '.csv'
                self.__generate_csv_or_txt(self.get_export_path(export_datafile), header, ';')
                self.output_format = '.gdt'
                self.__generate_gdt(self.get_export_path(export_datafile), header)
            elif self.output_format.lower() == '.csv':
                self.__generate_csv_or_txt(self.get_export_path(export_datafile), header, ';')
            elif self.output_format.lower() == '.txt':
                self.__generate_csv_or_txt(self.get_export_path(export_datafile), header, '\t')
            else:
                self.__generate_gdt(self.get_export_path(export_datafile), header)

    def enumerate_results(self):
        for element in Config.ELEMENTS_STATISTICS:
            yield self.get_name(element), getattr(self, element)

    def get_name(self, variable):
        match variable:
            case 'bankruptcy':
                return 'bankruptcies'
            case 'P':
                return 'prob_change_lender'
            case 'B':
                return 'bad_debt'
        return variable

    def get_data(self):
        result = pd.DataFrame()
        for variable_name, variable in self.enumerate_results():
            result[variable_name] = np.array(variable)
        return result

    def plot_pygrace(self, xx, yy_s, variable, title, export_datafile, x_label, y_label):
        from pygrace.project import Project
        plot = Project()
        graph = plot.add_graph()
        graph.title.text = title.capitalize().replace("_", ' ')
        for (yy, color, ticks, title_y) in yy_s:
            data = []
            if type(yy) == type(()):
                for i in range(len(yy[0])):
                    data.append((yy[0][i], yy[1][i]))
            else:
                for i in range(len(xx)):
                    data.append((xx[i], yy[i]))
            dataset = graph.add_dataset(data, legend=title_y)
            dataset.symbol.fill_color = color
        graph.xaxis.label.text = x_label
        graph.yaxis.label.text = y_label
        graph.set_world_to_limits()
        graph.autoscale()
        if export_datafile:
            if self.plot_format:
                plot.saveall(self.get_export_path(export_datafile, f"_{variable.lower()}{self.plot_format}"))

    def plot_pyplot(self, xx, yy_s, variable, title, export_datafile, x_label, y_label):
        if self.plot_format == '.agr':
            self.plot_pygrace(xx, yy_s, variable, title, export_datafile, x_label, y_label)
        else:
            plt.clf()
            plt.figure(figsize=(14, 6))
            for (yy, color, ticks, title_y) in yy_s:
                if type(yy) == type(()):
                    plt.plot(yy[0], yy[1], ticks, color=color, label=title_y, linewidth=0.2)
                else:
                    plt.plot(xx, yy, ticks, color=color, label=title_y)
            plt.xlabel(x_label)
            if y_label:
                plt.ylabel(y_label)
            plt.title(title.capitalize().replace("_", ' '))
            if len(yy_s) > 1:
                plt.legend()
            if export_datafile:
                if self.plot_format:
                    plt.savefig(self.get_export_path(export_datafile, f"_{variable.lower()}{self.plot_format}"))
            else:
                plt.show()
            plt.close()

    def plot_result(self, variable, title, export_datafile=None):
        xx = []
        yy = []
        for i in range(self.model.config.T):
            xx.append(i)
            yy.append(getattr(self, variable)[i])
        self.plot_pyplot(xx, [(yy, 'blue', '-', '')], variable, title, export_datafile, "Time", '')

    def get_plots(self, export_datafile):
        for variable in Config.ELEMENTS_STATISTICS:
            if Config.ELEMENTS_STATISTICS[variable]:  # True if they are going to be plot
                if f'plot_{variable}' in dir(Statistics):
                    eval(f'self.plot_{variable}(export_datafile)')
                else:
                    self.plot_result(variable, self.get_name(variable), export_datafile)

    def plot_P(self, export_datafile=None):
        xx = []
        yy = []
        yy_min = []
        yy_max = []
        yy_std = []

        for i in range(self.model.config.T):
            xx.append(i)
            yy.append(self.P[i])
            yy_min.append(self.P_min[i])
            yy_max.append(self.P_max[i])
            yy_std.append(self.P_std[i])
        self.plot_pyplot(xx, [(yy, 'blue', '-', 'Avg prob with $\\gamma$'),
                              (yy_min, "cyan", ':', "Max and min prob"),
                              (yy_max, "cyan", ':', ''),
                              (yy_std, "red", '-', "Std")
                              ],
                         'prob_change_lender', "Prob of change lender " + self.model.config.lender_change.describe(),
                         export_datafile, 'Time', '')


    def plot_best_lender(self, export_datafile=None):
        xx = []
        yy = []
        yy2 = []
        max_duration = 0
        final_best_lender = -1
        current_lender = -1
        current_duration = 0
        time_init = 0
        for i in range(self.model.config.T):
            xx.append(i)
            yy.append(self.best_lender[i] / self.model.config.N)
            yy2.append(self.best_lender_clients[i] / self.model.config.N)
            if current_lender != self.best_lender[i]:
                if max_duration < current_duration:
                    max_duration = current_duration
                    time_init = i - current_duration
                    final_best_lender = current_lender
                current_lender = self.best_lender[i]
                current_duration = 0
            else:
                current_duration += 1

        xx3 = []
        yy3 = []
        for i in range(time_init, time_init + max_duration):
            xx3.append(i)
            yy3.append(self.best_lender[i] / self.model.config.N)
        self.plot_pyplot(xx, [(yy, 'blue', '-', 'id'),
                              (yy2, "red", '-', "Num clients"),
                              ((xx3, yy3), "orange", '-', "")
                              ],
                         'best_lender', "Best Lender (blue) #clients (red)",
                         export_datafile,
                         f"Time (best lender={final_best_lender} at t=[{time_init}..{time_init + max_duration}])",
                         "Best Lender")



class Log:
    """
    The class acts as a logger and helpers to represent the data and evol from the Model.
    """
    logger = logging.getLogger("model")
    modules = []
    model = None
    logLevel = "ERROR"
    progress_bar = None
    graphical = False

    def __init__(self, model):
        self.model = model

    def define_gui(self, gui):
        self.graphical = gui.gooey

    def do_progress_bar(self, message, maximum):
        if self.graphical:
            self.progress_bar = Gui()
            self.progress_bar.progress_bar(message, maximum)
        else:
            from progress.bar import Bar
            self.progress_bar = Bar(message, max=maximum)

    @staticmethod
    def __format_number__(number):
        result = f"{number:5.2f}"
        while len(result) > 5 and result[-1] == "0":
            result = result[:-1]
        while len(result) > 5 and result.find('.') > 0:
            result = result[:-1]
        return result

    def __get_string_debug_banks__(self, details, bank):
        text = f"{bank.getId():10} C={Log.__format_number__(bank.C)} L={Log.__format_number__(bank.L)}"
        amount_borrowed = 0
        list_borrowers = " borrows=["
        for bank_i in bank.activeBorrowers:
            list_borrowers += self.model.banks[bank_i].getId(short=True) + ","
            amount_borrowed += bank.activeBorrowers[bank_i]
        if amount_borrowed:
            text += f" l={Log.__format_number__(amount_borrowed)}"
            list_borrowers = list_borrowers[:-1] + "]"
        else:
            text += "        "
            list_borrowers = ""
        text += f" | D={Log.__format_number__(bank.D)} E={Log.__format_number__(bank.E)}"
        if details and hasattr(bank, 'd') and bank.d and bank.l:
            text += f" l={Log.__format_number__(bank.d)}"
        else:
            text += "        "
        if details and hasattr(bank, 's') and bank.s:
            text += f" s={Log.__format_number__(bank.s)}"
        else:
            if details and hasattr(bank, 'd') and bank.d:
                text += f" d={Log.__format_number__(bank.d)}"
            else:
                text += "        "
        if bank.failed:
            text += f" FAILED "
        else:
            if details and hasattr(bank, 'd') and bank.d > 0:
                if bank.getLender() is None:
                    text += f" no lender"
                else:
                    text += f" lender{bank.getLender().getId(short=True)},r={bank.getLoanInterest():.2f}%"
            else:
                text += list_borrowers
        text += f" B={Log.__format_number__(bank.B)}" if bank.B else "        "
        return text

    def debug_banks(self, details: bool = True, info: str = ''):
        for bank in self.model.banks:
            if not info:
                info = "-----"
            self.info(info, self.__get_string_debug_banks__(details, bank))

    @staticmethod
    def get_level(option):
        try:
            return getattr(logging, option.upper())
        except AttributeError:
            logging.error(f" '--log' must contain a valid logging level and {option.upper()} is not.")
            sys.exit(-1)

    def debug(self, module, text):
        if self.modules == [] or module in self.modules:
            if text:
                self.logger.debug(f"t={self.model.t:03}/{module:6} {text}")

    def info(self, module, text):
        if self.modules == [] or module in self.modules:
            if text:
                self.logger.info(f" t={self.model.t:03}/{module:6} {text}")

    def error(self, module, text):
        if text:
            self.logger.error(f"t={self.model.t:03}/{module:6} {text}")

    def define_log(self, log: str, logfile: str = '', modules: str = '', script_name: str = "%(module)s"):
        self.modules = modules.split(",") if modules else []
        formatter = logging.Formatter('%(levelname)s-' + script_name + '- %(message)s')
        self.logLevel = Log.get_level(log.upper())
        self.logger.setLevel(self.logLevel)
        if logfile:
            if not os.path.dirname(logfile):
                logfile = f"{self.model.statistics.OUTPUT_DIRECTORY}/{logfile}"
            fh = logging.FileHandler(logfile, 'a', 'utf-8')
            fh.setLevel(self.logLevel)
            fh.setFormatter(formatter)
            self.logger.addHandler(fh)
        else:
            ch = logging.StreamHandler()
            ch.setLevel(self.logLevel)
            ch.setFormatter(formatter)
            self.logger.addHandler(ch)


class Model:
    """
    It contains the banks and has the logic to execute the simulation
        import interbank
        model = interbank.Model( )
        model.configure( param=x )
        model.forward()
        μ = model.get_current_fitness()
        model.set_policy_recommendation( ŋ=0.5 )
    or
        model.set_policy_recommendation( 1 ) --> equal to n=0.5, because values are int 0,1,2 ---> float 0,0.5,1

    If you want to forward() and backward() step by step, you should use:
        model.configure( backward=True )
        # t=4
        model.set_policy_recommendation( ŋ=0.5 )
        model.forward()
        result = model.get_current_fitness() # t=5
        model.backward() # t=4 again



    """
    banks = []  # An array of Bank with size Model.config.N
    t: int = 0  # current value of time, t = 0..Model.config.T
    eta: float = 1  # ŋ eta : current value of policy recommendation
    test = False  # it's true when we are inside a test
    default_seed: int = 20579  # seed for this simulation
    backward_enabled = False  # if true, we can execute backward()
    policy_changes = 0

    # if not None, we will debug at this instant i, entering in interactive mode
    debug = None

    # if not None, it should be a list of t in which we generate a graph with lenders, as i.e., [0,800]
    save_graphs = None
    save_graphs_results = []

    log = None
    statistics = None
    config = None
    export_datafile = None
    export_description = None

    policy_actions_translation = [0.0, 0.5, 1.0]

    generate_plots = True

    def __init__(self, **configuration):
        self.log = Log(self)
        self.statistics = Statistics(self)
        self.config = Config()
        if configuration:
            self.configure(**configuration)
        self.banks_copy = []

    def configure(self, **configuration):
        for attribute in configuration:
            if attribute.startswith('lc'): #TODO
                attribute = attribute.replace("lc_", "")
                if attribute == 'lc':
                    self.config.lender_change = determine_algorithm(configuration[attribute])
                    print(self.config.lender_change.__name__)
                else:
                    self.config.lender_change.set_parameter(attribute, configuration["lc_" + attribute])
            elif hasattr(self.config, attribute):
                current_value = getattr(self.config, attribute)
                if isinstance(current_value, int):
                    setattr(self.config, attribute, int(configuration[attribute]))
                else:
                    if isinstance(current_value, float):
                        setattr(self.config, attribute, float(configuration[attribute]))
                    else:
                        raise Exception(f"type of config {attribute} not allowed: {type(current_value)}")
            else:
                raise LookupError("attribute in config not found")
        self.initialize()

    def initialize(self, seed=None, dont_seed=False, save_graphs_instants=None,
                   export_datafile=None, export_description=None, generate_plots=True, output_directory=None):
        self.statistics.reset(output_directory=output_directory)
        if not dont_seed:
            random.seed(seed if seed else self.default_seed)
        self.save_graphs = save_graphs_instants
        self.banks = []
        self.t = 0
        if not self.config.lender_change:
            self.config.lender_change = determine_algorithm()
        self.policy_changes = 0
        if export_datafile:
            self.export_datafile = export_datafile
        if generate_plots:
            self.generate_plots = generate_plots
        self.export_description = str(self.config) if export_description is None else export_description
        for i in range(self.config.N):
            self.banks.append(Bank(i, self))
        self.config.lender_change.initialize_bank_relationships(self)

    def forward(self):
        self.initialize_step()
        if self.backward_enabled:
            self.banks_copy = copy.deepcopy(self.banks)
        self.do_shock("shock1")
        self.do_loans()
        self.log.debug_banks()
        self.statistics.compute_interest_loans_leverage()
        self.do_shock("shock2")
        self.do_repayments()
        self.log.debug_banks()
        if self.log.progress_bar:
            self.log.progress_bar.next()
        self.statistics.compute_liquidity()
        self.statistics.compute_credit_channels_and_best_lender()
        self.statistics.compute_fitness()
        self.statistics.compute_policy()
        self.statistics.compute_bad_debt()
        self.statistics.compute_rationing()
        self.setup_links()
        self.statistics.compute_probability_of_lender_change()
        self.log.debug_banks()
        if self.save_graphs is not None and (self.save_graphs == '*' or self.t in self.save_graphs):
            filename = self.statistics.get_graph(self.t)
            if filename:
                self.save_graphs_results.append(filename)
        if self.debug and self.t == self.debug:
            import code
            code.interact(local=locals())
        self.t += 1

    def backward(self):
        if self.backward_enabled:
            if self.t > 0:
                self.banks = self.banks_copy
                self.t -= 1
            else:
                raise IndexError('t=0 and no backward is possible')
        else:
            raise AttributeError('enable_backward() before')

    def do_debug(self, debug):
        self.debug = debug

    def enable_backward(self):
        self.backward_enabled = True

    def simulate_full(self, interactive=False):
        if interactive:
            self.log.do_progress_bar(f"Simulating t=0..{self.config.T}", self.config.T)
        for t in range(self.config.T):
            self.forward()

    def finish(self):
        if not self.test:
            self.statistics.export_data(export_datafile=self.export_datafile,
                                        export_description=self.export_description,
                                        generate_plots=self.generate_plots)
        summary = f"Finish: model T={self.config.T}  N={self.config.N}"
        if not self.__policy_recommendation_changed__():
            summary += f" ŋ={self.eta}"
        else:
            summary += " ŋ variate during simulation"
        self.log.info("*****", summary)
        self.statistics.create_gif_with_graphs(self.save_graphs_results)
        plt.close()
        return self.statistics.get_data()

    def set_policy_recommendation(self, n: int = None, eta: float = None, eta_1: float = None):
        if eta_1 is not None:
            n = round(eta_1)
        if n is not None and eta is None:
            if type(n) is int:
                eta = self.policy_actions_translation[n]
            else:
                eta = float(n)
        if self.eta != eta:
            self.log.debug("*****", f"eta(ŋ) changed to {eta}")
            self.policy_changes += 1
        self.eta = eta

    def limit_to_two_policies(self):
        self.policy_actions_translation = [0.0, 1.0]

    def __policy_recommendation_changed__(self):
        return self.policy_changes > 1

    def get_current_fitness(self):
        """
        Determines the current μ of the model (does the sum of all μ)
        :return:
        float:  Ʃ banks.μ
        """
        return self.statistics.fitness[self.t - 1 if self.t > 0 else 0]

    def get_current_credit_channels(self):
        """
        Determines the number of credits channels USED (each bank has a possible lender, but only if
        it needs it borrows money. This number represents how many banks have set up a credit with a lender
        :return:
        int
        """
        return self.statistics.credit_channels[self.t - 1 if self.t > 0 else 0]

    def get_current_liquidity(self):
        """
        Returns the liquidity (the sum of the liquidity)
        :return:
        float:  Ʃ banks.C
        """
        return self.statistics.liquidity[self.t - 1 if self.t > 0 else 0]

    def get_current_interest_rate(self):
        """
        Returns the interest rate (the average of all banks)
        :return:
        float:  Ʃ banks.ir / config.N
        """
        return self.statistics.interest_rate[self.t - 1 if self.t > 0 else 0]

    def get_current_interest_rate_info(self):
        """
        Returns a tuple with  : max ir, min_ir and avg
        :return:
        (float,float,float)
        """
        max_ir = 0
        min_ir = np.inf
        for bank in self.banks:
            bank_ir = bank.getLoanInterest()
            if bank_ir is not None:
                if max_ir < bank_ir:
                    max_ir = bank_ir
                if min_ir > bank_ir:
                    min_ir = bank_ir
        return max_ir, min_ir, self.get_current_interest_rate()

    def get_current_liquidity_info(self):
        """
        Returns a tuple with  : max C, min C and avg C
        :return:
        (float,float,float)
        """
        max_c = 0
        min_c = 1e6
        for bank in self.banks:
            bank_c = bank.C
            if max_c < bank_c:
                max_c = bank_c
            if min_c > bank_c:
                min_c = bank_c
        return max_c, min_c, self.get_current_liquidity()

    def get_current_bankruptcies(self):
        """
        Returns the number of bankruptcies in this step
        :return:
        int:  Ʃ failed banks
        """
        return self.statistics.bankruptcy[self.t - 1 if self.t > 0 else 0]

    def do_shock(self, which_shock):
        # (equation 2)
        for bank in self.banks:
            bank.newD = bank.D * (self.config.mi + self.config.omega * random.random())
            bank.incrD = bank.newD - bank.D
            bank.D = bank.newD
            if bank.incrD >= 0:
                bank.C += bank.incrD
                # if "shock1" then we can be a lender:
                if which_shock == "shock1":
                    bank.s = bank.C
                bank.d = 0  # it will not need to borrow
                if bank.incrD > 0:
                    self.log.debug(which_shock,
                                   f"{bank.getId()} wins ΔD={bank.incrD:.3f}")
            else:
                # if "shock1" then we cannot be a lender: we have lost deposits
                if which_shock == "shock1":
                    bank.s = 0
                if bank.incrD + bank.C >= 0:
                    bank.d = 0  # it will not need to borrow
                    bank.C += bank.incrD
                    self.log.debug(which_shock,
                                   f"{bank.getId()} loses ΔD={bank.incrD:.3f}, covered by capital")
                else:
                    bank.d = abs(bank.incrD + bank.C)  # it will need money
                    self.log.debug(which_shock,
                                   f"{bank.getId()} loses ΔD={bank.incrD:.3f} but has only C={bank.C:.3f}")
                    bank.C = 0  # we run out of capital
            self.statistics.incrementD[self.t] += bank.incrD

    def do_loans(self):
        for bank in self.banks:
            # decrement in which we should borrow
            if bank.d > 0:
                if bank.getLender() is None:
                    bank.l = 0
                    # new situation: the bank has no borrower, so we should firesale or die:
                    bank.doFiresalesL(bank.d, f"no lender for this bank", "loans")
                    bank.rationing = bank.d
                elif bank.getLender().d > 0:
                    # if the lender has no increment then NO LOAN could be obtained: we fire sale L:
                    bank.doFiresalesL(bank.d, f"lender {bank.getLender().getId(short=True)} has no money", "loans")
                    bank.rationing = bank.d
                    bank.l = 0
                else:
                    # if the lender can give us money, but not enough to cover the loan we need also fire sale L:
                    if bank.d > bank.getLender().s:
                        bank.doFiresalesL(bank.d - bank.getLender().s,
                                          f"lender.s={bank.getLender().s:.3f} but need d={bank.d:.3f}", "loans")
                        bank.rationing = bank.d - bank.getLender().s
                        # only if lender has money, because if it .s=0, all is obtained by fire sales:
                        if bank.getLender().s > 0:
                            bank.l = bank.getLender().s  # amount of loan (wrote in the borrower)
                            self.statistics.credit_channels[self.t] += 1
                            bank.getLender().activeBorrowers[
                                bank.id] = bank.getLender().s  # amount of loan (wrote in the lender)
                            bank.getLender().C -= bank.l  # amount of loan that reduces lender capital
                            bank.getLender().s = 0
                    else:
                        bank.l = bank.d  # amount of loan (wrote in the borrower)
                        self.statistics.credit_channels[self.t] += 1
                        bank.getLender().activeBorrowers[bank.id] = bank.d  # amount of loan (wrote in the lender)
                        bank.getLender().s -= bank.d  # the loan reduces our lender's capacity to borrow to others
                        bank.getLender().C -= bank.d  # amount of loan that reduces lender capital
                        self.log.debug("loans",
                                       f"{bank.getId()} new loan l={bank.d:.3f} from {bank.getLender().getId()}")

            # the shock can be covered by own capital
            else:
                bank.l = 0
                if len(bank.activeBorrowers) > 0:
                    list_borrowers = ""
                    amount_borrowed = 0
                    for bank_i in bank.activeBorrowers:
                        list_borrowers += self.banks[bank_i].getId(short=True) + ","
                        amount_borrowed += bank.activeBorrowers[bank_i]
                    self.log.debug("loans", f"{bank.getId()} has a total of {len(bank.activeBorrowers)} loans with " +
                                   f"[{list_borrowers[:-1]}] of l={amount_borrowed}")

    def do_repayments(self):
        # first all borrowers must pay their loans:
        for bank in self.banks:
            if bank.l > 0:
                loan_profits = bank.getLoanInterest() * bank.l
                loan_to_return = bank.l + loan_profits
                # (equation 3)
                if loan_to_return > bank.C:
                    we_need_to_sell = loan_to_return - bank.C
                    bank.C = 0
                    bank.paid_loan = bank.doFiresalesL(
                        we_need_to_sell,
                        f"to return loan and interest {loan_to_return:.3f} > C={bank.C:.3f}",
                        "repay")
                # the firesales of line above could bankrupt the bank, if not, we pay "normally" the loan:
                else:
                    bank.C -= loan_to_return
                    bank.E -= loan_profits
                    bank.paid_loan = bank.l
                    bank.l = 0
                    bank.getLender().s -= bank.l  # we reduce the  's' => the lender could have more loans
                    bank.getLender().C += loan_to_return  # we return the loan and it's profits
                    bank.getLender().E += loan_profits  # the profits are paid as E
                    self.log.debug(
                        "repay",
                        f"{bank.getId()} pays loan {loan_to_return:.3f} (E={bank.E:.3f},C={bank.C:.3f}) to lender" +
                        f" {bank.getLender().getId()} (ΔE={loan_profits:.3f},ΔC={bank.l:.3f})")

        # now  when ΔD<0 it's time to use Capital or sell L again
        # (now we have the loans cancelled, or the bank bankrputed):
        for bank in self.banks:
            if bank.d > 0 and not bank.failed:
                bank.doFiresalesL(bank.d, f"fire sales due to not enough C", "repay")

        for bank in self.banks:
            bank.activeBorrowers = {}
            if bank.failed:
                bank.replaceBank()
        self.log.debug("repay", f"this step ΔD={self.statistics.incrementD[self.t]:.3f} and " +
                       f"failures={self.statistics.bankruptcy[self.t]}")

    def initialize_step(self):
        for bank in self.banks:
            bank.B = 0
            bank.rationing = 0
        # self.config.lender_change.initialize_step(self)
        if self.t == 0:
            self.log.debug_banks()

    def setup_links(self):
        # (equation 5)
        # p = probability borrower not failing
        # c = lending capacity
        # h = borrower haircut (leverage of bank respect to the maximum)
        maxE = max(self.banks, key=lambda k: k.E).E
        maxC = max(self.banks, key=lambda k: k.C).C
        for bank in self.banks:
            bank.p = bank.E / maxE
            bank._lambda = bank.l / bank.E
            bank.incrD = 0

        max_lambda = max(self.banks, key=lambda k: k._lambda)._lambda
        for bank in self.banks:
            bank.h = bank._lambda / max_lambda if max_lambda > 0 else 0
            bank.A = bank.C + bank.L  # bank.L / bank.λ + bank.D

        # determine c (lending capacity) for all other banks (to whom give loans):
        for bank in self.banks:
            bank.c = []
            for i in range(self.config.N):
                c = 0 if i == bank.id else (1 - self.banks[i].h) * self.banks[i].A
                bank.c.append(c)

        # (equation 6)
        minr = sys.maxsize
        lines = []

        for bank_i in self.banks:
            line1 = ""
            line2 = ""

            bank_i.asset_i = 0
            bank_i.asset_j = 0

            for j in range(self.config.N):
                try:
                    if j == bank_i.id:
                        bank_i.rij[j] = 0
                    else:
                        if self.banks[j].p == 0 or bank_i.c[j] == 0:
                            bank_i.rij[j] = self.config.r_i0
                        else:
                            bank_i.rij[j] = (self.config.ji * bank_i.A -
                                             self.config.phi * self.banks[j].A -
                                             (1 - self.banks[j].p) *
                                             (self.config.xi * self.banks[j].A - bank_i.c[j])) \
                                            / (self.banks[j].p * bank_i.c[j])

                            bank_i.asset_i += self.config.ji * bank_i.A
                            bank_i.asset_j += self.config.phi * self.banks[j].A
                            bank_i.asset_j += (1 - self.banks[j].p)
                        if bank_i.rij[j] < 0:
                            bank_i.rij[j] = self.config.r_i0
                # the first t=1, maybe t=2, the shocks have not affected enough to use L (only C), so probably
                # L and E are equal for all banks, and so max_lambda=anyλ and h=1 , so cij=(1-1)A=0, and r division
                # by zero -> solution then is to use still r_i0:
                except ZeroDivisionError:
                    bank_i.rij[j] = self.config.r_i0

                line1 += f"{bank_i.rij[j]:.3f},"
                line2 += f"{bank_i.c[j]:.3f},"
            if lines:
                lines.append("  |" + line2[:-1] + "|   |" +
                             line1[:-1] + f"| {bank_i.getId(short=True)} h={bank_i.h:.3f},λ={bank_i._lambda:.3f} ")
            else:
                lines.append("c=|" + line2[:-1] + "| r=|" +
                             line1[:-1] + f"| {bank_i.getId(short=True)} h={bank_i.h:.3f},λ={bank_i._lambda:.3f} ")

            bank_i.r = np.sum(bank_i.rij) / (self.config.N - 1)
            bank_i.asset_i = bank_i.asset_i / (self.config.N - 1)
            bank_i.asset_j = bank_i.asset_j / (self.config.N - 1)

            if bank_i.r < minr:
                minr = bank_i.r

        if self.config.N < 10:
            for line in lines:
                self.log.debug("links", f"{line}")
        self.log.debug("links",
                       f"maxE={maxE:.3f} maxC={maxC:.3f} max_lambda={max_lambda:.3f} minr={minr:.3f} ŋ={self.eta:.3f}")

        # (equation 7)
        loginfo = loginfo1 = ""
        for bank in self.banks:
            # bank.μ mu
            bank.mu = self.eta * (bank.C / maxC) + (1 - self.eta) * (minr / bank.r)
            loginfo += f"{bank.getId(short=True)}:{bank.mu:.3f},"
            loginfo1 += f"{bank.getId(short=True)}:{bank.r:.3f},"
        if self.config.N <= 10:
            self.log.debug("links", f"μ=[{loginfo[:-1]}] r=[{loginfo1[:-1]}]")

        self.config.lender_change.step_setup_links(self)
        for bank in self.banks:
            self.log.debug("links", self.config.lender_change.change_lender(self, bank, self.t))

In [ ]:

class Bank:
    """
    It represents an individual bank of the network, with the logic of interaction between it and the interbank system
    """

    def getLender(self):
        if self.lender is None:
            return None
        else:
            return self.model.banks[self.lender]

    def getLoanInterest(self):
        if self.lender is None:
            return None
        else:
            # only we take in account if the bank has a lender active, so the others will return always None
            return self.model.banks[self.lender].rij[self.id]

    def getId(self, short: bool = False):
        init = "bank#" if not short else "#"
        if self.failures > 0:
            return f"{init}{self.id}.{self.failures}"
        else:
            return f"{init}{self.id}"

    def __init__(self, new_id, bank_model):
        self.id = new_id
        self.model = bank_model
        self.failures = 0
        self.rationing = 0
        self.lender = None
        self.__assign_defaults__()

    def __assign_defaults__(self):
        self.L = self.model.config.L_i0
        self.C = self.model.config.C_i0
        self.D = self.model.config.D_i0
        self.E = self.model.config.E_i0
        self.mu = 0  # fitness of the bank:  estimated later
        self.l = 0  # amount of loan done:  estimated later
        self.s = 0  # amount of loan received: estimated later
        self.B = 0  # bad debt: estimated later
        self.failed = False
        # identity of the lender
        self.lender = self.model.config.lender_change.new_lender(self.model, self)
        self.activeBorrowers = {}
        self.asset_i = 0
        self.asset_j = 0

    def replaceBank(self):
        self.failures += 1
        self.__assign_defaults__()

    def __doBankruptcy__(self, phase):
        self.failed = True
        self.model.statistics.bankruptcy[self.model.t] += 1
        recovered_in_fire_sales = self.L * self.model.config.ro  # we firesale what we have
        recovered = recovered_in_fire_sales - self.D  # we should pay D to clients
        if recovered < 0:
            recovered = 0
        if recovered > self.l:
            recovered = self.l

        badDebt = self.l - recovered  # the fire sale minus paying D: what the lender recovers
        if badDebt > 0:
            self.paidLoan = recovered
            self.getLender().B += badDebt
            self.getLender().E -= badDebt
            self.getLender().C += recovered
            self.model.log.debug(phase, f"{self.getId()} bankrupted (fire sale={recovered_in_fire_sales:.3f}," +
                                 f"recovers={recovered:.3f},paidD={self.D:.3f})(lender{self.getLender().getId(short=True)}" +
                                 f".ΔB={badDebt:.3f},ΔC={recovered:.3f})")
        else:
            # self.l=0 no current loan to return:
            if self.l > 0:
                self.paidLoan = self.l  # the loan was paid, not the interest
                self.getLender().C += self.l  # lender not recovers more than loan if it is
                self.model.log.debug(phase, f"{self.getId()} bankrupted (lender{self.getLender().getId(short=True)}" +
                                     f".ΔB=0,ΔC={recovered:.3f}) (paidD={self.l:.3f})")
        self.D = 0
        # the loan is not paid correctly, but we remove it
        if self.getLender() and self.id in self.getLender().activeBorrowers:
            self.getLender().s -= self.l
            del self.getLender().activeBorrowers[self.id]

    def doFiresalesL(self, amountToSell, reason, phase):
        costOfSell = amountToSell / self.model.config.ro
        recoveredE = costOfSell * (1 - self.model.config.ro)
        if costOfSell > self.L:
            self.model.log.debug(phase,
                                 f"{self.getId()} impossible fire sale sellL={costOfSell:.3f} > L={self.L:.3f}: {reason}")
            return self.__doBankruptcy__(phase)
        else:
            self.L -= costOfSell
            self.E -= recoveredE

            if self.L <= self.model.config.alfa:
                self.model.log.debug(phase,
                                     f"{self.getId()} new L={self.L:.3f} makes bankruptcy of bank: {reason}")
                return self.__doBankruptcy__(phase)
            else:
                if self.E <= self.model.config.alfa:
                    self.model.log.debug(phase,
                                         f"{self.getId()} new E={self.E:.3f} makes bankruptcy of bank: {reason}")
                    return self.__doBankruptcy__(phase)
                else:
                    self.model.log.debug(phase,
                                         f"{self.getId()} fire sale sellL={amountToSell:.3f} at cost {costOfSell:.3f} reducing" +
                                         f"E={recoveredE:.3f}: {reason}")
                    return amountToSell


class Gui:
    gooey = False

    def progress_bar(self, message, maximum):
        print(message)
        self.maximum = maximum
        self.current = 1

    def next(self):
        self.current += 1
        print("progress: {}%".format(self.current / self.maximum * 100))
        sys.stdout.flush()

    def parser(self):
        try:
            import interbank_gui
            parser = interbank_gui.get_interactive_parser()
        except:
            parser = None
        if parser is None:
            parser = argparse.ArgumentParser()
        else:
            self.gooey = True
        return parser


class Utils:
    """
    Auxiliary class to encapsulate the use of the model
    """

    @staticmethod
    def __extract_t_values_from_arg__(param):
        if param is None:
            return None
        else:
            t = []
            if param == '*':
                return '*'
            else:
                for str_t in param.split(","):
                    t.append(int(str_t))
                    if t[-1] > Config.T or t[-1] < 0:
                        raise ValueError(f"{t[-1]} greater than Config.T or below 0")
                return t

    @staticmethod
    def run_interactive():
        """
            Run interactively the model
        """
        global model
        gui = Gui()
        parser = gui.parser()
        parser.add_argument("--log", default='ERROR', help="Log level messages (ERROR,DEBUG,INFO...)")
        parser.add_argument("--modules", default=None, help=f"Log only this modules (separated by ,)")
        parser.add_argument("--logfile", default=None, help="File to send logs to")
        parser.add_argument("--save", default=None, help=f"Saves the output of this execution")
        parser.add_argument("--graph", default=None,
                            help=f"List of t in which save the network config (* for all)")
        parser.add_argument("--gif_graph", default=False,
                            type=bool,
                            help=f"If --graph, then also an animated gif with all graphs ")
        parser.add_argument("--n", type=int, default=Config.N, help=f"Number of banks")
        parser.add_argument("--debug", type=int, default=None,
                            help="Stop and enter in debug mode after at this time")
        parser.add_argument("--eta", type=float, default=Model.eta, help=f"Policy recommendation")
        parser.add_argument("--t", type=int, default=Config.T, help=f"Time repetitions")
        parser.add_argument("--lc_p", type=float, default=None,
                            help=f"For Erdos-Renyi bank lender's change value of p=0.0x")
        parser.add_argument("--lc_m", type=int, default=None, #todo
                            help=f"For Preferential bank lender's change value of graph grade m")
        parser.add_argument("--lc", type=str, default="default",
                            help="Bank lender's change method (?=list)")
        parser.add_argument("--lc_ini_graph_file", type=str, default=None,
                            help="Load a graph in json networkx.node_link_data() format")
        parser.add_argument("--plot_format", type=str, default="none",
                            help="Generate plots with the specified format (svg,png,pdf,gif,agr)")
        parser.add_argument("--output_format", type=str, default="gdt",
                            help="File extension for data (gdt,txt,csv,both)")
        parser.add_argument("--output", type=str, default=None,
                            help="Directory where to store the results")
        args = parser.parse_args()
        if args.t != model.config.T:
            model.config.T = args.t
        if args.n != model.config.N:
            model.config.N = args.n
        if args.eta != model.eta:
            model.eta = args.eta
        if args.debug:
            model.do_debug(args.debug)
        model.config.lender_change = determine_algorithm(args.lc)
        model.config.lender_change.set_parameter("p", args.lc_p)
        model.config.lender_change.set_parameter("m", args.lc_m)
        model.config.lender_change.set_initial_graph_file(args.lc_ini_graph_file)
        model.log.define_log(args.log, args.logfile, args.modules)
        model.log.define_gui(gui)
        model.statistics.define_output_format(args.output_format)
        model.statistics.set_gif_graph(args.gif_graph)
        model.statistics.define_plot_format(args.plot_format)
        Utils.run(args.save, Utils.__extract_t_values_from_arg__(args.graph),
                  output_directory=args.output,
                  interactive=(args.log == 'ERROR' or args.logfile is not None))

    @staticmethod
    def run(save=None, save_graph_instants=None, interactive=False, output_directory=None):
        global model
        if not save_graph_instants and Config.GRAPHS_MOMENTS:
            save_graph_instants = Config.GRAPHS_MOMENTS
        model.initialize(export_datafile=save, save_graphs_instants=save_graph_instants,
                         output_directory=output_directory)
        model.simulate_full(interactive=interactive)
        return model.finish()

    @staticmethod
    def is_notebook():
        try:
            # noinspection PyBroadException
            __IPYTHON__
            return get_ipython().__class__.__name__ != "SpyderShell"
        except NameError:
            return False

    @staticmethod
    def is_spyder():
        # noinspection PyBroadException
        try:
            return get_ipython().__class__.__name__ == "SpyderShell"
        except NameError:
            return False

In [ ]:


model = Model()
if Utils.is_notebook():
    model.statistics.OUTPUT_DIRECTORY = '/content'
    model.statistics.output_format = 'csv'
    # if we are running in a Notebook:
    Utils.run(save="results")
else:
    # if we are running interactively:
    if __name__ == "__main__":
        Utils.run_interactive()